In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RRP3DRQ3X to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-129696
Azure region: southcentralus
Subscription id: dafd62fe-51d3-4671-af50-6bac2f001b5d
Resource group: aml-quickstarts-129696


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
amlcompute_cluster_name= "cpu-cluster"

#Verify that cluster does not already exist
try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [41]:
#Define RunConfig for the compute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler (continuous)
ps = RandomParameterSampling ({
    "--C":uniform(0.5,1.0), #equivalent to the value of k
    "--max_iter":choice(25,50,75,100)
    #max_iter between 1 to 999
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

#Define primary metric goal
#goal=primary_metric_goal("MAXIMIZE")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='training',
compute_target=aml_compute,vm_size="STANDARD_D2_V2",vm_priority="lowpriority",entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
    max_concurrent_runs=4
)

In [42]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
hyperdrive_run=experiment.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
auto_url="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###